# Tutorial: Climate Networks

The objective of this tutorial is to introduce climate networks and explain and illustrate their application with the __pyunicorn__ package. First some theoretical background for understanding general climate networks will be given and then some methods provided by `pyunicorn.climate.ClimateNetwork` will be illustrated. An introduction and application of coupled climate networks will follow. For a detailed discussion and further references, please consult __[Donges et al., 2015](https://aip.scitation.org/doi/10.1063/1.4934554)__, on which this tutorial is based. 

## Introduction

_Climate networks (CN)_ are a way to apply complex network theory to the climate system, by assuming that each node represents a varying dynamical system. Of interest is then the collective behaviour of these interacting dynamical system and the structure of the resulting network. This approach was first introduced by __[Tsonis and Roebber, 2004](https://www.sciencedirect.com/science/article/abs/pii/S0378437103009646)__.

Climate network analysis is a versatile approach for investigating climatological data and can be used as a complementary method to classical techniques from multivariate statistics. The approach allows for the analysis of single fields of climatological time series, e.g. surface air temperature observed on a grid, or even two or more fields. It has been succesfully applied in many cases, for example to dynamics and predictability of the El Niño Phenomenon \[__[Radebach et al., 2013](https://arxiv.org/abs/1310.5494)__\].

## Theory of Climate Networks (CN)

Climate networks are a typical application of _functional networks_, which allow to study the dynamical relationships between subsystems of a high-dimensional complex system by constructing networks from it. The package provides classes for the construction and analysis of such networks, representing the statistical interdependency structure within and between fields of time series using various similarity measures.

### Similarity Measures for Time Series